# ARIMA, LSTM, and Bayesian Ridge — with Hybrid LSTM+Transformer Features

This notebook trains three forecasters on a single target ticker:

1. **ARIMA / SARIMAX** (classical)
2. **LSTM** (sequence model)
3. **Bayesian Ridge** with **hybrid features** from **LSTM** and a **Transformer Encoder**

**Data**: `adj_close_wide.parquet` (5 years of daily adjusted close, wide by ticker). We model **next-day return**.

Outputs:
- `artifacts/lstm_savedmodel` — Keras LSTM forecaster
- `artifacts/transformer_savedmodel` — Keras Transformer forecaster (used for embeddings)
- `artifacts/bayes_hybrid.joblib` — Bayesian Ridge on hybrid embeddings (+ optional lag features)
- `artifacts/sarimax.pkl` — SARIMAX with exogenous hybrid features
- `artifacts/meta.json` — parameters & metrics

You can later use these artifacts in your local UI.

In [ ]:
#@title 0) Setup
!pip -q install pandas pyarrow numpy scikit-learn statsmodels joblib tensorflow matplotlib --upgrade
import os, json, math, pickle
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.pipeline import Pipeline
import joblib
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from statsmodels.tsa.statespace.sarimax import SARIMAX
print('Setup complete. TF:', tf.__version__)

In [ ]:
#@title 1) Paths and parameters
ADJ_CLOSE_WIDE = "/content/drive/MyDrive/ai_stock_watcher/data/curated/adj_close_wide.parquet"  #@param {type:"string"}
TARGET_TICKER = "AAPL"  #@param {type:"string"}
WINDOW = 90  #@param {type:"integer"}
TEST_SPLIT_FRACTION = 0.2  #@param {type:"number"}
ART_DIR = "/content/drive/MyDrive/ai_stock_watcher/artifacts"  #@param {type:"string"}
Path(ART_DIR).mkdir(parents=True, exist_ok=True)
print('ART_DIR =', ART_DIR)

In [ ]:
#@title 2) Load prices and create returns
prices = pd.read_parquet(ADJ_CLOSE_WIDE).sort_index().asfreq('B')
if TARGET_TICKER not in prices.columns:
    raise SystemExit(f"{TARGET_TICKER} not found in columns. Available: {list(prices.columns)[:6]} ...")
ret = prices[TARGET_TICKER].pct_change().dropna()
display(ret.tail())
print('Date range:', ret.index.min(), '→', ret.index.max(), 'rows=', len(ret))

In [ ]:
#@title 3) Window the series → supervised dataset (WINDOW → predict next return)
def make_windows(series: pd.Series, window: int):
    s = series.values.astype('float32')
    X, y, idx_end = [], [], []
    for i in range(window, len(s)):
        X.append(s[i-window:i])     # window ending at time i-1
        y.append(s[i])              # predict return at time i (next day)
        idx_end.append(series.index[i-1])  # index of the window end (aligns exog later)
    X = np.stack(X)
    y = np.array(y, dtype='float32')
    idx_end = pd.to_datetime(idx_end)
    return X, y, idx_end

raw_X, raw_y, idx_end = make_windows(ret, WINDOW)
print('Raw window tensor:', raw_X.shape, 'targets:', raw_y.shape)

In [ ]:
#@title 4) Train/val/test split (time-based) and scaling
n = len(raw_X)
n_train = int(n * (1 - TEST_SPLIT_FRACTION))
n_val = int(n_train * 0.1)  # 10% of train for validation
n_train_final = n_train - n_val

X_train, y_train = raw_X[:n_train_final], raw_y[:n_train_final]
X_val, y_val     = raw_X[n_train_final:n_train], raw_y[n_train_final:n_train]
X_test, y_test   = raw_X[n_train:], raw_y[n_train:]
idx_end_train    = idx_end[:n_train]
idx_end_test     = idx_end[n_train:]

# Scale returns using train stats, apply to all windows
sc = StandardScaler()
sc.fit(X_train.reshape(len(X_train), -1))
def scale_windows(X):
    shp = X.shape
    X2 = sc.transform(X.reshape(len(X), -1)).reshape(shp)
    return X2

Xs_train = scale_windows(X_train)
Xs_val   = scale_windows(X_val)
Xs_test  = scale_windows(X_test)
print('Splits:', Xs_train.shape, Xs_val.shape, Xs_test.shape)

In [ ]:
#@title 5) LSTM forecaster
tf.random.set_seed(42)
inputs = keras.Input(shape=(WINDOW, 1))
x = layers.LSTM(64, return_sequences=True, name='lstm1')(inputs)
x = layers.Dropout(0.2)(x)
x = layers.LSTM(32, name='lstm2')(x)
emb_lstm = layers.Dense(32, activation='tanh', name='lstm_embedding')(x)  # embedding for hybrid features
out = layers.Dense(1, name='out')(emb_lstm)
lstm_model = keras.Model(inputs, out, name='lstm_forecaster')
lstm_model.compile(optimizer=keras.optimizers.Adam(1e-3), loss='mse')

hist = lstm_model.fit(
    Xs_train[..., None], y_train,
    validation_data=(Xs_val[..., None], y_val),
    epochs=8, batch_size=32, verbose=1,
    callbacks=[keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)]
)
yhat_lstm = lstm_model.predict(Xs_test[..., None], verbose=0).ravel()
rmse_lstm = mean_squared_error(y_test, yhat_lstm, squared=False)
mae_lstm  = mean_absolute_error(y_test, yhat_lstm)
dir_lstm  = (np.sign(yhat_lstm) == np.sign(y_test)).mean()
print(f"LSTM → RMSE={rmse_lstm:.6f}  MAE={mae_lstm:.6f}  DirAcc={dir_lstm:.3f}")

# Save LSTM
lstm_path = os.path.join(ART_DIR, 'lstm_savedmodel')
lstm_model.save(lstm_path)
print('Saved LSTM:', lstm_path)

In [ ]:
#@title 6) Transformer forecaster (for embeddings)
def positional_encoding(length, d_model):
    pos = np.arange(length)[:, None]
    i = np.arange(d_model)[None, :]
    angle_rates = 1 / np.power(10000, (2*(i//2))/np.float32(d_model))
    angle_rads = pos * angle_rates
    sines = np.sin(angle_rads[:, 0::2])
    cosines = np.cos(angle_rads[:, 1::2])
    pe = np.zeros((length, d_model))
    pe[:, 0::2] = sines
    pe[:, 1::2] = cosines
    return tf.constant(pe, dtype=tf.float32)

class AddPositionalEncoding(layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def build(self, input_shape):
        self.len = input_shape[1]
        self.d_model = input_shape[2]
        self.pe = positional_encoding(self.len, self.d_model)
    def call(self, x):
        return x + self.pe

def transformer_block(x, num_heads=2, dff=64, dropout=0.1):
    # Layer norm + MHA
    attn_in = layers.LayerNormalization(epsilon=1e-6)(x)
    attn_out = layers.MultiHeadAttention(num_heads=num_heads, key_dim=x.shape[-1])(attn_in, attn_in)
    x = layers.Add()([x, layers.Dropout(dropout)(attn_out)])
    # FFN
    ffn_in = layers.LayerNormalization(epsilon=1e-6)(x)
    ffn_out = layers.Dense(dff, activation='relu')(ffn_in)
    ffn_out = layers.Dense(x.shape[-1])(ffn_out)
    x = layers.Add()([x, layers.Dropout(dropout)(ffn_out)])
    return x

tf.random.set_seed(42)
tin = keras.Input(shape=(WINDOW, 1))
x = layers.Dense(32)(tin)  # project to model dim
x = AddPositionalEncoding()(x)
x = transformer_block(x, num_heads=2, dff=64, dropout=0.1)
x = transformer_block(x, num_heads=2, dff=64, dropout=0.1)
x = layers.GlobalAveragePooling1D()(x)
emb_tr = layers.Dense(32, activation='tanh', name='tr_embedding')(x)  # embedding for hybrid features
tout = layers.Dense(1, name='out')(emb_tr)
tr_model = keras.Model(tin, tout, name='transformer_forecaster')
tr_model.compile(optimizer=keras.optimizers.Adam(1e-3), loss='mse')

hist2 = tr_model.fit(
    Xs_train[..., None], y_train,
    validation_data=(Xs_val[..., None], y_val),
    epochs=8, batch_size=32, verbose=1,
    callbacks=[keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)]
)
yhat_tr = tr_model.predict(Xs_test[..., None], verbose=0).ravel()
rmse_tr = mean_squared_error(y_test, yhat_tr, squared=False)
mae_tr  = mean_absolute_error(y_test, yhat_tr)
dir_tr  = (np.sign(yhat_tr) == np.sign(y_test)).mean()
print(f"Transformer → RMSE={rmse_tr:.6f}  MAE={mae_tr:.6f}  DirAcc={dir_tr:.3f}")

# Save Transformer
tr_path = os.path.join(ART_DIR, 'transformer_savedmodel')
tr_model.save(tr_path)
print('Saved Transformer:', tr_path)

In [ ]:
#@title 7) Extract hybrid embeddings for all samples
lstm_embedder = keras.Model(lstm_model.input, lstm_model.get_layer('lstm_embedding').output)
tr_embedder   = keras.Model(tr_model.input,   tr_model.get_layer('tr_embedding').output)

def embed_all(Xs):
    e1 = lstm_embedder.predict(Xs[..., None], verbose=0)
    e2 = tr_embedder.predict(Xs[..., None],   verbose=0)
    return e1, e2

e1_train, e2_train = embed_all(Xs_train)
e1_val,   e2_val   = embed_all(Xs_val)
e1_test,  e2_test  = embed_all(Xs_test)

# Concatenate embeddings (+ optional last k raw lags as features)
def last_k_lags(Xs, k=10):
    return Xs[:, -k:]

lag_k = 10
lag_trn = last_k_lags(Xs_train, lag_k)
lag_val = last_k_lags(Xs_val,   lag_k)
lag_tst = last_k_lags(Xs_test,  lag_k)

Z_train = np.concatenate([e1_train, e2_train, lag_trn], axis=1)
Z_val   = np.concatenate([e1_val,   e2_val,   lag_val], axis=1)
Z_test  = np.concatenate([e1_test,  e2_test,  lag_tst], axis=1)
print('Hybrid feature shapes:', Z_train.shape, Z_val.shape, Z_test.shape)

In [ ]:
#@title 8) Bayesian Ridge on hybrid features
pipe_bayes = Pipeline([
    ("scaler", StandardScaler()),
    ("bayes", BayesianRidge(compute_score=True))
])
pipe_bayes.fit(np.vstack([Z_train, Z_val]), np.concatenate([y_train, y_val]))
yhat_bayes = pipe_bayes.predict(Z_test)
rmse_b = mean_squared_error(y_test, yhat_bayes, squared=False)
mae_b  = mean_absolute_error(y_test, yhat_bayes)
dir_b  = (np.sign(yhat_bayes) == np.sign(y_test)).mean()
print(f"Bayesian Ridge (hybrid) → RMSE={rmse_b:.6f}  MAE={mae_b:.6f}  DirAcc={dir_b:.3f}")

joblib.dump(pipe_bayes, os.path.join(ART_DIR, 'bayes_hybrid.joblib'))
print('Saved Bayesian Ridge:', os.path.join(ART_DIR, 'bayes_hybrid.joblib'))

In [ ]:
#@title 9) SARIMAX with exogenous hybrid features (optional but included)
# Align exogenous features with target times: our windows end at t, target is y at t+1.
exog_full = np.concatenate([e1_train, e2_train, last_k_lags(Xs_train, lag_k)], axis=1)
exog_full = np.vstack([exog_full, np.concatenate([e1_val, e2_val, last_k_lags(Xs_val, lag_k)], axis=1)])
exog_test = np.concatenate([e1_test, e2_test, last_k_lags(Xs_test, lag_k)], axis=1)

# Build pandas index-aligned series for SARIMAX
idx_train_full = idx_end_train  # end-of-window dates for train+val
idx_test_full  = idx_end_test

# Align y's index to next day
y_train_full = np.concatenate([y_train, y_val])
y_train_idx = idx_train_full + pd.tseries.offsets.BDay(1)
y_test_idx  = idx_test_full  + pd.tseries.offsets.BDay(1)
y_train_series = pd.Series(y_train_full, index=y_train_idx)
y_test_series  = pd.Series(y_test,       index=y_test_idx)

# Shift exog so its index matches y index (forecast time)
exog_train_df = pd.DataFrame(exog_full, index=idx_train_full)
exog_test_df  = pd.DataFrame(exog_test,  index=idx_test_full)
exog_train_shift = exog_train_df.copy(); exog_train_shift.index = y_train_idx
exog_test_shift  = exog_test_df.copy();  exog_test_shift.index  = y_test_idx

# Fit SARIMAX on returns with exogenous features
sarimax = SARIMAX(y_train_series, order=(1,0,1), seasonal_order=(0,0,0,0), exog=exog_train_shift,
                  enforce_stationarity=False, enforce_invertibility=False)
sarimax_res = sarimax.fit(disp=False)
fc = sarimax_res.forecast(steps=len(y_test_series), exog=exog_test_shift)
rmse_s = mean_squared_error(y_test_series.values, fc.values, squared=False)
mae_s  = mean_absolute_error(y_test_series.values, fc.values)
dir_s  = (np.sign(fc.values) == np.sign(y_test_series.values)).mean()
print(f"SARIMAX(exog) → RMSE={rmse_s:.6f}  MAE={mae_s:.6f}  DirAcc={dir_s:.3f}")

with open(os.path.join(ART_DIR, 'sarimax.pkl'), 'wb') as f:
    pickle.dump(sarimax_res, f)
print('Saved SARIMAX:', os.path.join(ART_DIR, 'sarimax.pkl'))

In [ ]:
#@title 10) Compare model performance and plot test predictions
metrics = {
    'lstm':   {'rmse': float(rmse_lstm), 'mae': float(mae_lstm), 'dir_acc': float(dir_lstm)},
    'transformer': {'rmse': float(rmse_tr), 'mae': float(mae_tr), 'dir_acc': float(dir_tr)},
    'bayes_hybrid': {'rmse': float(rmse_b), 'mae': float(mae_b), 'dir_acc': float(dir_b)},
    'sarimax_exog': {'rmse': float(rmse_s), 'mae': float(mae_s), 'dir_acc': float(dir_s)}
}
print(json.dumps(metrics, indent=2))

plt.figure()
plt.plot(y_test_series.index, y_test_series.values, label='Actual')
plt.plot(y_test_series.index, yhat_lstm, label='LSTM')
plt.plot(y_test_series.index, yhat_tr, label='Transformer')
plt.plot(y_test_series.index, yhat_bayes, label='Bayes Hybrid')
plt.plot(y_test_series.index, fc.values, label='SARIMAX exog')
plt.title(f"{TARGET_TICKER} — Test Next-Day Returns (model comparison)")
plt.legend(); plt.xlabel('Date'); plt.ylabel('Return'); plt.show()

In [ ]:
#@title 11) Save meta.json
meta = {
    'target': TARGET_TICKER,
    'window': int(WINDOW),
    'splits': {
        'train_end_index': int(len(Xs_train)-1),
        'val_end_index': int(len(Xs_train)+len(Xs_val)-1)
    },
    'metrics': metrics
}
with open(os.path.join(ART_DIR, 'meta.json'), 'w') as f:
    json.dump(meta, f, indent=2)
print('Saved meta.json')